# trackStatistics
Filter dataset, background correction, trackStatistics uses trackAnalyis files as input. <br/>

This notebook as to be reloaded before analysing a new group of data!

## Load cell.h5 files

Choose/insert a directory for the cell files to be searched in (e.g. F:\data_folder). All *.h5 files in the directory and subdirectories will be loaded for the analysis.

In [1]:
from pySPT.widgets.widgetNotebooks import init_filter_notebook
from pySPT.widgets.widgetNotebooks import init_track_stats_widget_arrangement
from pySPT.widgets.widgetNotebooks import init_save_track_stats
from pySPT.widgets.widgetNotebooks import init_save_filtered_analysis
from pySPT.widgets.widgetNotebooks import init_save_filtered_trc
from pySPT.widgets import widgetLoadHdf5
from pySPT.widgets import loadHdf5
from pySPT.widgets import saveStatistics
from pySPT.widgets import saveFiltered
from pySPT.Analysis import cell
from pySPT.Analysis import trajectory
from pySPT.Analysis import coverSlip
from pySPT.Analysis import trajectoryStatistics
from tqdm import tqdm_notebook as tqdm
import os
import warnings
warnings.filterwarnings("ignore")
widget_load_hdf5 = widgetLoadHdf5.WidgetLoadHdf5(bin_size="0.1", x_range="2", y_range="0.5")  # adjust the default parameters
load_hdf5 = loadHdf5.LoadHdf5()
cover_slip = coverSlip.CoverSlip()
track_stats = trajectoryStatistics.TrajectoryStatistics()
display(widget_load_hdf5.dir_box, widget_load_hdf5.dir_button)
widget_load_hdf5.dir_button.on_click(widget_load_hdf5.open_dir)
widget_load_hdf5.dir_box.observe(widget_load_hdf5.change_dir_box)

Text(value='', description='Directory', placeholder='directory to be searched in', style=DescriptionStyle(desc…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [2]:
def initialize(event):
    init_filter_notebook(cover_slip, widget_load_hdf5, load_hdf5, is_cell=True)
    track_stats.cells = cover_slip.cells
    track_stats.cell_trajectories = cover_slip.cell_trajectories
    track_stats.cell_sizes = [cell.size for cell in cover_slip.cells]
    track_stats.create_filtered_framework()
display(widget_load_hdf5.init_cells_button)
widget_load_hdf5.init_cells_button.on_click(initialize)

Button(description='initialize', style=ButtonStyle(), tooltip='initialize objects')

## Load background.h5 files

Choose/insert a directory for the background files to be searched in (e.g. F:\CS2\background). All *.h5 files in the directory will be loaded for the analysis. Background correction is recommended but optional.

In [3]:
display(widget_load_hdf5.dir_box_bg, widget_load_hdf5.dir_button_bg)
widget_load_hdf5.dir_box_bg.observe(widget_load_hdf5.change_dir_box_bg)
widget_load_hdf5.dir_button_bg.on_click(widget_load_hdf5.open_dir_bg)

Text(value='', description='Directory', placeholder='directory to be searched in', style=DescriptionStyle(desc…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [4]:
def initialize(event):
    init_filter_notebook(cover_slip, widget_load_hdf5, load_hdf5, is_cell=False)
    track_stats.backgrounds = cover_slip.backgrounds
    track_stats.background_trajectories = cover_slip.background_trajectories
    track_stats.bg_sizes = [background.size for background in cover_slip.backgrounds]
display(widget_load_hdf5.init_background_button)
widget_load_hdf5.init_background_button.on_click(initialize)

Button(description='initialize', style=ButtonStyle(), tooltip='initialize objects')

## Filter for parameters & display results

Filter the dataset by min/max allowed values and diffusion states (checked boxes = included). Multiple filtering of a dataset is possible without restarting the notebook. <br/>

The results are visualized as global diffusion histogram, MSD-plot and distributions with average cell values. <br/>
*Diffusion histogram:* The normalized frequency (per area) in % is plotted against the diffusion coefficients (log bins). Diffusion coefficients <= 0 are neglected, because the logarithm is not defined in this interval. Choose a *bin size*.<br/>
*MSD-plot*: The average MSD-curves are plotted per diffusion type. Choose a *x range* in seconds and a *y range* for the plot to display. None displays the entire value range.<br/>
*Cell distributions:* The average is build per cell and the distributions displayed as violin plots.

In [5]:
def filter(event):
    widget_load_hdf5.create_clear_output()
    print("Filter options")
    display(widget_load_hdf5.min_length_box, widget_load_hdf5.max_length_box, widget_load_hdf5.min_D_box, widget_load_hdf5.max_D_box, widget_load_hdf5.immob_type_check_box, widget_load_hdf5.confined_type_check_box, widget_load_hdf5.free_type_check_box, widget_load_hdf5.analyse_not_successful_check_box)
    print("Display options")
    display(widget_load_hdf5.bin_size_box, widget_load_hdf5.MSD_delta_t_n, widget_load_hdf5.MSD_y_lim, widget_load_hdf5.filter_button)
    track_stats.run_statistics(widget_load_hdf5.min_length_box.value, widget_load_hdf5.max_length_box.value, widget_load_hdf5.min_D_box.value, widget_load_hdf5.max_D_box.value, widget_load_hdf5.immob_type_check_box.value,  widget_load_hdf5.confined_type_check_box.value, widget_load_hdf5.free_type_check_box.value, widget_load_hdf5.analyse_not_successful_check_box.value)
    track_stats.run_diffusion_histogram(widget_load_hdf5.bin_size_box.value, widget_load_hdf5.MSD_delta_t_n.value, widget_load_hdf5.MSD_y_lim.value, 0)

print("Filter options")
display(widget_load_hdf5.min_length_box, widget_load_hdf5.max_length_box, widget_load_hdf5.min_D_box, widget_load_hdf5.max_D_box, widget_load_hdf5.immob_type_check_box, widget_load_hdf5.confined_type_check_box, widget_load_hdf5.free_type_check_box, widget_load_hdf5.analyse_not_successful_check_box)
print("Display options")
display(widget_load_hdf5.bin_size_box, widget_load_hdf5.MSD_delta_t_n, widget_load_hdf5.MSD_y_lim, widget_load_hdf5.filter_button)
widget_load_hdf5.filter_button.on_click(filter)

Filter options


Text(value='', description='Trajectory', placeholder='min length', style=DescriptionStyle(description_width='i…

Text(value='', description='Trajectory', placeholder='max length', style=DescriptionStyle(description_width='i…

Text(value='', description='Diffusion coefficient', placeholder='min value', style=DescriptionStyle(descriptio…

Text(value='', description='Diffusion coefficient', placeholder='max value', style=DescriptionStyle(descriptio…

...

Display options


Text(value='0.1', description='bin size', placeholder='size for log10(D) histogram', style=DescriptionStyle(de…

Text(value='2', description='x range', placeholder='x range in seconds', style=DescriptionStyle(description_wi…

Text(value='0.5', description='y range', placeholder='y limit of MSD plot', style=DescriptionStyle(description…

Button(description='filter & plot', style=ButtonStyle(), tooltip='apply filter and display global information'…

## Calculate dynamic localization error 

For each cell, a dynamic localization error based on the filtered trajectories is calculated. This error can be used for the 'Minimal detectable D' parameter in the trackAnalysis notebook.

In [6]:
def calc(event):
    widget_load_hdf5.create_clear_output()
    display(widget_load_hdf5.calc_sigma_dyn_button)
    for cell_idx in range(len(track_stats.cell_trajectories_filtered)):
        print("{}: {} \u03BCm".format(track_stats.cells[cell_idx].name, track_stats.sigma_dyns[cell_idx]))
display(widget_load_hdf5.calc_sigma_dyn_button)
widget_load_hdf5.calc_sigma_dyn_button.on_click(calc)

Button(description='calc', style=ButtonStyle(), tooltip='dynamic localization error')

## Choose trajectory to plot

In [7]:
def dropdown(event):
    widget_load_hdf5.cell_options = widget_load_hdf5.get_cell_names(cover_slip.cells, track_stats.cell_trajectories_filtered)
widget_load_hdf5.filter_button.on_click(dropdown)
display(widget_load_hdf5.drop_down_cells)

Dropdown(description='Cell:', options=(), value=None)

In [8]:
def dropdown(event):      
    for cell in cover_slip.cells:
        if cell.name == widget_load_hdf5.drop_down_cells.value:
            target_cell = cover_slip.cells.index(cell)
            widget_load_hdf5.trajectory_options = track_stats.cell_trajectories_filtered_index[target_cell]
            widget_load_hdf5.drop_down_trajectories.options = widget_load_hdf5.trajectory_options
widget_load_hdf5.drop_down_cells.observe(dropdown)
widget_load_hdf5.filter_button.on_click(dropdown)
display(widget_load_hdf5.drop_down_trajectories)

Dropdown(description='Trajectory:', options=(), value=None)

In [9]:
def plot_trajectory(event):
    widget_load_hdf5.create_clear_output()
    display(widget_load_hdf5.plot_button)
    cover_slip.plot_trajectory(widget_load_hdf5.drop_down_cells.value, widget_load_hdf5.drop_down_trajectories.value)
display(widget_load_hdf5.plot_button)
widget_load_hdf5.plot_button.on_click(plot_trajectory)

Button(description='plot', style=ButtonStyle(), tooltip='plot chosen trajectory')

## Save Analysis

Choose a directory, folder name and a file name for the statistics h5 file. If you want to save the filtered dataset, check the box. The filtered dataset will be saved in the same folder as the statistics h5 file.<br/>

Please make sure to visualize the plots before saving.

In [10]:
display(widget_load_hdf5.save_dir_button, widget_load_hdf5.save_dir_box, widget_load_hdf5.save_folder_name_box, widget_load_hdf5.save_name_box, widget_load_hdf5.filtered_dataset_checkbox, widget_load_hdf5.Dplot_checkbox)
widget_load_hdf5.save_dir_button.on_click(widget_load_hdf5.save_open_dir)
widget_load_hdf5.save_dir_box.observe(widget_load_hdf5.change_save_dir_box)

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

Text(value='', description='directory', placeholder='directory for filtered data', style=DescriptionStyle(desc…

Text(value='211014_filtered', description='folder name', placeholder='name of folder', style=DescriptionStyle(…

Text(value='statistics', description='file name', placeholder='name for statistics .h5 file', style=Descriptio…

Checkbox(value=True, description='Save filtered dataset')

Checkbox(value=True, description='Save global plots')

In [11]:
def save(event):
    widget_load_hdf5.create_clear_output()
    display(widget_load_hdf5.save_button)
    h5_stats = saveStatistics.SaveStatistics()

    names = [i.name for i in track_stats.cells]
    if len(names) != len(set(names)):
        print("Error: A target file name existed multiple times. Please make sure to name each file individually to ensure proper analysis and saving. Currently loaded file names:", names)
    else:
        if os.path.exists(widget_load_hdf5.save_dir_box.value + "\\" + widget_load_hdf5.save_folder_name_box.value):
            print("Directory already exsists. Please choose an other directory or folder name.")
        else:
            os.makedirs(widget_load_hdf5.save_dir_box.value + "\\" + widget_load_hdf5.save_folder_name_box.value)
            if widget_load_hdf5.save_dir_box.value:
                init_save_track_stats(h5_stats, track_stats, widget_load_hdf5.save_dir_box.value, widget_load_hdf5.save_folder_name_box.value, widget_load_hdf5.save_name_box.value)
                if widget_load_hdf5.filtered_dataset_checkbox.value:
                    track_stats.filter_cell_trc()
                    for cell_index in tqdm(range(len(track_stats.cells))):
                        if track_stats.cell_trajectories_filtered[cell_index]:
                            init_save_filtered_analysis(cover_slip, cell_index, track_stats, widget_load_hdf5.save_dir_box.value, widget_load_hdf5.save_folder_name_box.value)
                    print("The filtered dataset is saved.")
                if widget_load_hdf5.hmm_trc_checkbox.value:
                    print("The trc files of the filtered dataset will be saved.")
                    track_stats.filter_cell_trc()
                    for cell_index in tqdm(range(len(track_stats.cells))):
                        if track_stats.cell_trajectories_filtered[cell_index]:
                            init_save_filtered_trc(track_stats, widget_load_hdf5.save_dir_box.value, widget_load_hdf5.save_folder_name_box.value)
                if widget_load_hdf5.Dplot_checkbox.value and track_stats.diff_fig:
                    track_stats.save_diff_fig(widget_load_hdf5.save_dir_box.value, widget_load_hdf5.save_folder_name_box.value)
            else:
                print("Please choose a directory")
display(widget_load_hdf5.save_button) 
widget_load_hdf5.save_button.on_click(save)

Button(description='save', style=ButtonStyle(), tooltip='save statistics')